###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2020 L.A. Barba, N.C. Clementi

# Untangle any waveform

Welcome to a new lesson of the _Engineering Computations_ series, where we use computing as our instrument to discover the world. In this lesson, we explore information in wave form, using sound as a prototypical example. We aim to bring to light how engineers and scientists are able to analyze this kind of data, which has a myriad interesting applications. Let's go!

## What is sound?

Physically, _sound_ means travelling vibrations in any medium: air, water, metal, anything that can support oscillations. When your vocal chords emit sound, they are literally vibrating and passing on that vibration to the air around them. 

Imagine pressure vibrations in the air like longitudinal waves on a slinky, as shown on this gif (you can watch the full [video](https://www.youtube.com/watch?v=GIkeGBXqWW0) on YouTube).

![Longitudinal Waves on a Slinky](https://i.makeagif.com/media/5-30-2017/ZChjoc.gif)

#### Compression/expansion longitudinal wave on a slinky.

Every loop of the spiral just oscillates around a position of equilibrium, while the _wave_ of compressions and expansions travels the length of the slinky. In the air, sound also travels as longitudinal compression/rarefaction waves.

### Waveforms

A waveform is a schematic to understand sound waves. A fundamental sound is represented by a _sine wave_. Do explore the interactive guide to [waveforms](https://pudding.cool/2018/02/waveforms/) online: it presents both visually and audibly the concepts of displacement amplitude, wave frequency and more.

When you play on the middle of a piano the note of A, the piano string vibrates at 440 oscillations per second (Hz). That is the _frequency_ of the corresponding sine wave:

$$y(t) = A \sin(2\pi \, f \, t + \theta)$$

where $A$ is the wave amplitude, $f$ the frequency, and $\theta$ is the phase, i.e., the angle value of the sine at $t=0$. The corresponding angular frequency is $~\omega=2\pi f$, in radians per second. 

Let's visualize this function using our favorite Python libraries. We first create a `time` array holding the discrete values of time to evaluate the function at. The array is defined using the length of time from zero to `length_sec` (in seconds) and a _sampling rate_, the number of time samples per second. Study the code below.

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [ ]:
# set fonts for plot lables and axes
pyplot.rc('font', family='serif', size='16')

In [ ]:
sample_rate = 44100  #Hz
length_sec = 2       #seconds
time = numpy.linspace(0, length_sec, sample_rate * length_sec)  

In [ ]:
freq = 440 #Hz
y = numpy.sin(2 * numpy.pi * freq * time) 

The sampling rate used above, [44100 Hz](https://en.wikipedia.org/wiki/44,100_Hz) (44.1 kHz), is the standard value for digital audio recordings going back to the 1970s. It is the sampling rate used in CDs, and in .mp3 audio files. 

A fundamental theorem in the field of digital signal processing states that to capture all the information of a continuous signal, the sampling rate needs to be more than twice the maximum frequency. It is the **Nyquist–Shannon sampling theorem**. Since our ears can perceive sound in a frequency range of 20–20,000 Hz (20 kHz), industry chose 44.1 kHz for digital audio. 

Let's look at our sine waveform for the A4 note.

In [ ]:
pyplot.figure(figsize=(8,3))
pyplot.plot(time[:1000], y[:1000])
pyplot.xlabel('time')
pyplot.title('Sine wave with a 440 Hz frequency, $A=1$');

### Play with audio in Python

We can _play_ the sound from our sine waveform using the [display](https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html) tools of IPython. Check out the documentation for details. When you use a NumPy array as the data, it needs the argument `rate` corresponding to the sampling rate.

In [ ]:
from scipy.io import wavfile
from IPython.display import Audio

In [ ]:
Audio(y, rate=sample_rate)

Isn't that cool? Let's now write a custom function that takes as arguments the frequency, duration of sound (and an optional phase angle) and it returns the time array and the signal, using 44.1 kHz as the sampling rate.

In [ ]:
def sine_signal(f, s, phase=0):
    '''Computes a sine signal of frquency f and duration s
    using a sample rate of 44.1kHz, and returns it along with 
    the time array
    
    Arguments:
    ----------
    f: float, frequency
    s: float, duration of the audio in seconds
    phase: float, optional phase in radians 
    
    Returns:
    --------
    wave: array, sinusoidal wave array of frequency f
    time: array, time samples
    
    '''
    samp_rate = 44100 #Hz
    time = numpy.linspace(0, s, samp_rate * s)  
    
    wave = numpy.sin(2 * numpy.pi * f * time + phase)
    
    return wave, time

Let's create two sine waves of 2-second duration with frequencies of 440 Hz and 294 Hz, corresponding to the notes A4 and D4 on the piano, and save them to Python variables. On the second line, we save only the signal using the index `[0]`. 

In [ ]:
A440, t2 = sine_signal(f=440, s=2)
D294 = sine_signal(f=294, s=2)[0] 

Now play the sound for the D note alone, and for the sum of the two notes. Note that we can do this because the two signals have the same length and sampling rate!

In [ ]:
Audio(D294, rate=sample_rate)

In [ ]:
Audio(D294+A440, rate=sample_rate)

In [ ]:
f, (ax1, ax2, ax3) = pyplot.subplots(3, 1, sharex=True, figsize=(12.0, 9.0))

ax1.plot(t2[:1000], (A440+D294)[:1000], c='C0', label='A440 + D294')

ax2.plot(t2[:1000], D294[:1000], c='C1', label='D294')
ax2.set_ylabel('Amplitude')

ax3.plot(t2[:1000], A440[:1000], c='C2', label='A440')

ax3.set_xlabel('time [s]')

#plot all legends in one line
[ax.legend(loc='upper right') for ax in (ax1, ax2, ax3)];

##### Exercise

- Create two new sound waves of 2-second duration and frequencies of 349Hz and 523Hz, and name them F349 and C523, respectively. 
- Create the audio widget to plays the sound that results from the sum of all the frequencies we have created.
- Plot all the sound waves and the sum of all of them in aplot like the one above.

## References

1. Josh Comeau (n.d.), [Waveforms](https://pudding.cool/2018/02/waveforms/), an interactive and visual guide.

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())